# Beispiel 4 Hochzeit

## Problemdaten:

Neun Personen sollen auf drei Tische verteilt werden, sodass die Sympathie-Werte der Personen pro Tisch maximiert wird.

### Sympathie-Werte

|           | Anna | Berti | Carl | Dieter | Emil | Franz | Gerd | Hanna | Ilse |
|:---------:|:----:|:-----:|:----:|:------:|:----:|:-----:|:----:|:-----:|:----:|
|    Anna   |   0  |   6   |   4  |   10   |   1  |   2   |   6  |   2   |   4  |
|   Berti   |   6  |   0   |   7  |    3   |   6  |   1   |   5  |   10  |   5  |
|    Carl   |   4  |   7   |   0  |   10   |   6  |   3   |  10  |   1   |   4  |
|   Dieter  |  10  |   3   |  10  |    0   |   4  |   6   |   9  |   6   |  10  |
|    Emil   |   1  |   6   |   6  |    4   |   0  |   10  |   1  |   2   |   4  |
|   Franz   |   2  |   1   |   3  |    6   |  10  |   0   |   9  |   7   |   7  |
|    Gerd   |   6  |   5   |  10  |    9   |   1  |   9   |   0  |   1   |   9  |
|   Hanna   |   2  |   10  |   1  |    6   |   2  |   7   |   1  |   0   |   8  |
|    Ilse   |   4  |   5   |   4  |   10   |   4  |   7   |   9  |   8   |   0  |

### Tischgrößen

Die drei Tische haben die Größe: 4, 3 und 2 Plätze

In [ ]:
import gurobipy as gp
from gurobipy import GRB

## Mengen - Definition

In [ ]:
numberOfDesks = 3
numberOfPersons = 9

desks = range(numberOfDesks)
persons = range(numberOfPersons)

## Parameter Eingabe

Sympathie-Werte:

In [ ]:
sympathy = [[0, 6, 4, 10, 1, 2, 6, 2, 4],
            [6, 0, 7, 3, 6, 1, 5, 10, 5],
            [4, 7, 0, 10, 6, 3, 10, 1, 4],
            [10, 3, 10, 0, 4, 6, 9, 6, 10],
            [1, 6, 6, 4, 0, 10, 1, 2, 4],
            [2, 1, 3, 6, 10, 0, 9, 7, 7],
            [6, 5, 10, 9, 1, 9, 0, 1, 9],
            [2, 10, 1, 6, 2, 7, 1, 0, 8],
            [4, 5, 4, 10, 4, 7, 9, 8, 0]]

Tischgrößen:

In [ ]:
capacity = [4, 3, 2]

## Initialisierung des Modells

In [ ]:
m = gp.Model()

## Initialisierung der Variablen

In [ ]:
x = m.addVars(numberOfPersons, numberOfDesks, vtype=GRB.BINARY, name="x")

## Definition der Zielfunktion

In [ ]:
m.setObjective(
    gp.quicksum(
        gp.quicksum(
            gp.quicksum(
                sympathy[person1][person2] * x[person1, desk] * x[person2, desk]
            for desk in desks)
        for person2 in persons if person2 != person1)
    for person1 in persons),
GRB.MAXIMIZE)

## Hinzufügen von Nebenbedingungen

In [ ]:
for desk in desks:
    m.addConstr(gp.quicksum(x[person, desk] for person in persons) <= capacity[desk], name="ConstrDesk"+str(desk))

#m.addConstrs((x.sum('*',desk) <= capacity[desk] for desk in desks), name="ConstrDesk")

for person in persons:
    m.addConstr(gp.quicksum(x[person, desk] for desk in desks) == 1, name="ConstrPerson"+str(person))

#m.addConstrs((x.sum(person,'*') == 1 for person in persons), name="ConstrPerson")

## Optimierung des Modells

In [ ]:
m.optimize()

## Ergebnisausgabe

In [ ]:
m.printAttr('ObjVal')

In [ ]:
m.printAttr('X')

## Intelligente Ausgabe

Defintion der Namen:

In [ ]:
names = ["Anna", "Berti", "Carl", "Dieter", "Emil", "Franz", "Gerd", "Hanna", "Ilse"]

In [ ]:
print("Sitzverteilung\n")
for person in persons:
    for desk in desks:
        if x[person, desk].X > 0:
            print(names[person], "\tsitzt am Tisch", desk)

In [ ]:
for desk in desks:
    print("\nPersonen am Tisch", desk)
    for person1 in range(numberOfPersons-1):
        for person2 in range(person1+1, numberOfPersons):
            if x[person1, desk].X * x[person2, desk].X > 0:
                print(names[person1], "\tund", names[person2], "\thaben den Sympathiewert:", sympathy[person1][person2])